In [ ]:
pip install pandas scikit-learn torch transformers datasets


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted
from joblib import dump, load
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

In [ ]:
df=pd.read_csv("/content/final dataset.csv")


In [ ]:
pip install unidecode

In [ ]:
df.tail(130)

,Query,Dialect,Query Real Answer
291,बिचारा घोळ्यावर बसून पोट्ट्याईचा नाच मूटुमूटु ...,Varhadi,बिचारा घोड्यावर बसून मुलांचा नाच शांतपणे बघत होता
292,जाय बरं थ्या घोळीच्या टिंबाटिंबावर येक लिचोंडा...,Varhadi,जा आणि त्या घोळातून एक लिंबाची फांदी घेऊन ये
293,त्याच्या वावरात पेरलेला सोला तीन मयन्यानंतर चा...,Varhadi,त्याच्या शेतात पेरलेला मका तीन महिन्यांनंतर चा...
294,नाईतं म्हनांन तुमी ह्या वकील कुठुंन आनला ईचिभीन,Varhadi,नाहितर तुम्ही विचाराल की मी हा वकील कुठून आणला...
295,मंग मंघानी इंधन घ्याले गेलतो तव्हा पाह्यली थे ...,Varhadi,मी इंधन तपासणार होतो मग काही वेळापूर्वीच कोण आ...
...,...,...,...
416,मला लवकर झोपायचे हाय,Satara,मला लवकर झोपायचे आहे
417,तो कामावर गेलाय,Satara,तो कामावर गेला आहे
418,मी नवीन घर घेतल य,Satara,मी नवीन घर घेतले आहे
419,आजची हवा खूप चांगली हाय,Satara,आज हवा खूप चांगली आहे


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Query              421 non-null    object
 1   Dialect            421 non-null    object
 2   Query Real Answer  421 non-null    object
dtypes: object(3)
memory usage: 10.0+ KB


In [ ]:
df['Dialect']=df['Dialect'].fillna('Ahirani')

In [ ]:
df['Dialect'].unique()

array(['Ahirani', 'konkani', 'Varhadi', 'Satara'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Query              421 non-null    object
 1   Dialect            421 non-null    object
 2   Query Real Answer  421 non-null    object
dtypes: object(3)
memory usage: 10.0+ KB


In [ ]:
df.columns

Index(['Query', 'Dialect', 'Query Real Answer'], dtype='object')

In [ ]:
# Features: 'query' (text)
X = df['Query']

# Targets: both categorical columns
y = df[['Dialect', 'Query Real Answer']]

# Quick check
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (421,)
y shape: (421, 2)


In [ ]:
f
# Split: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check sizes
print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

Train size: 336
Test size: 85


In [ ]:
# Pipeline: Text to numbers -> Multi-output classifier
model = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Handles text in 'query'
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train on train data
model.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


In [ ]:
X_train

,Query
131,देवाक लागीं म्हाका संभाळ करता.
31,रातोरात देव कसा झाया
84,सग्ळी जागावर तात्या हजर र्हायना...
289,माह्या त माह्यावरच इसवास नाई बसे
408,आम्ही एकत्र जेवतो
...,...
71,बबन्या: हाई काय ई राह्यनु हार लीसन! घ्या वं बह...
106,दुपारी जेवणानं पहिले खाऊन मी थोडी विश्रांती घेतली
270,मंगूभौ म्हनून वाज देल्ला पन ह्या शटर खोलाले तय...
348,फकड्या गावच्या चौकात आहे


In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# Accuracy for each output
dialect_acc = accuracy_score(y_test['Dialect'], y_pred[:, 0])
answer_acc = accuracy_score(y_test['Query Real Answer'], y_pred[:, 1])
print(f"Dialect accuracy: {dialect_acc * 100:.2f}%")
print(f"Real answer accuracy: {answer_acc * 100:.2f}%")

# Detailed reports
print("\nDialect Report:")
print(classification_report(y_test['Dialect'], y_pred[:, 0]))
print("\nReal Answer Report:")
print(classification_report(y_test['Query Real Answer'], y_pred[:, 1]))

Dialect accuracy: 51.76%
Real answer accuracy: 0.00%

Dialect Report:
              precision    recall  f1-score   support

     Ahirani       0.75      0.47      0.58        32
      Satara       0.60      0.55      0.57        22
     Varhadi       0.36      0.25      0.30        16
     konkani       0.38      0.87      0.53        15

    accuracy                           0.52        85
   macro avg       0.52      0.53      0.49        85
weighted avg       0.57      0.52      0.51        85


Real Answer Report:
                                                                                                            precision    recall  f1-score   support

                                                                                                 
रुबाबदार       0.00      0.00      0.00       0.0
                                                                             घराच्या बाहेरील मोकळ्या जागेत       0.00      0.00      0.00       0.0
                             

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [ ]:
# Example: Predict dialect for user-entered text

# Ask user for their query
user_query = input("Enter your query: ")

# Convert it to pandas Series (model expects this format)
new_queries = pd.Series([user_query])

# Predict
predictions = model.predict(new_queries)

# Display result
for query, pred in zip(new_queries, predictions):
    print(f"\nQuery: {query}")
    print(f"Predicted Dialect: {pred[0]}")
    print(f"Predicted Real Answer: {pred[1]}\n")


Enter your query: तुला लगिन व्हायेल का?	

Query: तुला लगिन व्हायेल का?	
Predicted Dialect: Varhadi
Predicted Real Answer: लग्नानंतर तो निघून जाऊ लागला

